In [3]:
import tushare as ts
import csv
import time
import pandas as pd
pro = ts.pro_api('35d8848b876df93910413e8936c40745d7b7da42553ae73920862cd9')

In [24]:
start_date = '20200101'
end_date = '20200301'
ts_code = '000001.SZ'
year='2020'

### 1. 股票数据获取（Tushare）

#### 1.1 股票基本信息

In [4]:
stock_basic = pro.stock_basic(list_status='L', fields='ts_code, symbol, name, industry')

# 重命名字段（便于后续导入neo4j）
basic_rename = {'ts_code': 'TS代码', 'symbol': '股票代码', 'name': '股票名称', 'industry': '行业'}
stock_basic.rename(columns=basic_rename, inplace=True)

# 保存为stock_basic.csv
stock_basic.to_csv('financial_data\\stock_basic.csv', encoding='gbk')

stock_basic.head()

,TS代码,股票代码,股票名称,行业
0,000001.SZ,000001,平安银行,银行
1,000002.SZ,000002,万科A,全国地产
2,000004.SZ,000004,国华网安,软件服务
3,000005.SZ,000005,ST星源,环境保护
4,000006.SZ,000006,深振业A,区域地产


#### 1.2 股票Top10股东信息

In [6]:
holders = pd.DataFrame(columns=('ts_code', 'ann_date', 'end_date', 'holder_name', 'hold_amount', 'hold_ratio'))

top10_holders = pro.top10_holders(ts_code=ts_code, start_date=start_date, end_date=end_date)

top10_holders.head()

,ts_code,ann_date,end_date,holder_name,hold_amount,hold_ratio
0,000001.SZ,20200421,20200331,中国平安保险(集团)股份有限公司-集团本级-自有资金,9.618540e+09,49.56
1,000001.SZ,20200421,20200331,香港中央结算有限公司(陆股通),1.669875e+09,8.60
2,000001.SZ,20200421,20200331,中国平安人寿保险股份有限公司-自有资金,1.186100e+09,6.11
3,000001.SZ,20200421,20200331,中国平安人寿保险股份有限公司-传统-普通保险产品,4.404787e+08,2.27
4,000001.SZ,20200421,20200331,中国证券金融股份有限公司,4.292327e+08,2.21


#### 1.3 股票概念信息

In [8]:
concept_details = pd.DataFrame(columns=('id', 'concept_name', 'ts_code', 'name'))

# concept_detail = pro.concept_detail(id='TS0')  # 该接口 TOKEN 受限
    
# concept_detail.head()

#### 1.4 股票公告信息

In [12]:
# notices = pro.anns(ts_code=ts_code, start_date=start_date, end_date=end_date, year=year)  # 该接口 TOKEN 受限

# notices.to_csv("financial_data\\notices\\"+str(code)+".csv",encoding='utf_8_sig',index=False)

# notices.head()

#### 1.5 财经新闻信息

In [129]:
news = pro.news(src='sina', start_date='2018-11-21 09:00:00', end_date='2018-11-22 10:10:00')
news.head()

,datetime,content,title
0,2018-11-22 10:09:00,【北京：10月末非金融企业贷款增速创历史同期新高 】据中国人民银行营业管理部11月22日披露...,
1,2018-11-22 10:08:29,【湖北省积极推动企业向科创板靠拢，年底将向上交所推荐一批优质企业】湖北省上市指导中心相关人士...,
2,2018-11-22 10:08:04,【上海：加快专利审查速度 进一步缩短专利授权周期】上海市人民政府发布关于加快本市高新技术企业...,
3,2018-11-22 10:04:35,【生意社：稀土价格走势或将上涨】11月20日稀土指数为344点，与上一日持平，较2011年1...,
4,2018-11-22 10:03:40,【上海：加快建设具有全球影响力的科技创新中心 做大做强高新技术产业】上海市人民政府发布关于加...,


#### 1.6 概念信息

In [19]:
# concept = pro.concept()  # 该接口 TOKEN 受限

# concept.head()

#### 1.7 沪股通成分和深股通成分信息 

In [20]:
sh = pro.hs_const(hs_type='SH')  # 获取沪股通成分

sz = pro.hs_const(hs_type='SZ')  # 获取深股通成分

sh.head()

,ts_code,hs_type,in_date,out_date,is_new
0,601628.SH,SH,20141117,None,1
1,601099.SH,SH,20141117,None,1
2,601808.SH,SH,20141117,None,1
3,601107.SH,SH,20141117,None,1
4,601880.SH,SH,20141117,None,1


#### 1.8 股票价格信息

In [29]:
end_date = '20200201'  # 使用更短数据目的是1.9绘制走势图数值不重叠

price = pro.query('daily', ts_code=ts_code, start_date=start_date, end_date=end_date)

price.head()

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20200123,15.92,15.92,15.39,15.54,16.09,-0.55,-3.4183,1100592.07,1723394.336
1,000001.SZ,20200122,15.92,16.16,15.71,16.09,16.00,0.09,0.5625,719464.91,1150933.398
2,000001.SZ,20200121,16.34,16.34,15.93,16.00,16.45,-0.45,-2.7356,896603.10,1442171.431
3,000001.SZ,20200120,16.43,16.61,16.35,16.45,16.39,0.06,0.3661,746074.75,1226464.649
4,000001.SZ,20200117,16.38,16.55,16.35,16.39,16.33,0.06,0.3674,605436.69,995909.007


#### 1.9 股票价格走势图

In [34]:
from pyecharts.charts import Line
from pyecharts import options as opts
import numpy as np

price = pro.query('daily', ts_code=ts_code, start_date=start_date, end_date=end_date)
(
    Line()
    .add_xaxis(xaxis_data=list(price['trade_date'])[::-1])
    .add_yaxis(series_name="收盘价",y_axis=list(price['close'])[::-1],symbol="circle")
#     .add_yaxis(series_name="开盘价",y_axis=list(price['open'])[::-1],symbol="circle")
#     .add_yaxis(series_name="最高价",y_axis=list(price['high'])[::-1],symbol="circle")
#     .add_yaxis(series_name="最低价",y_axis=list(price['low'])[::-1],symbol="circle")
    .set_global_opts(title_opts=opts.TitleOpts(title="价格走势图"))
    .render_notebook()
)

#### 1.10 基本面数据

In [110]:
stock_company = pro.stock_company(exchange='SZSE', fields='ts_code,chairman,manager,secretary,reg_capital,setup_date,province')
stock_company.head()

,ts_code,chairman,manager,secretary,reg_capital,setup_date,province
0,300952.SZ,王咸华,王咸华,张武芬,14492.7653,20040415,江苏
1,300268.SZ,汤捷,汤捷,杨振刚,17420.0000,20030508,湖南
2,300447.SZ,陈祥楼,何亮,孙璐,29106.9255,20010929,江苏
3,300451.SZ,葛航,张吕峥,胡燕,119260.8104,19971210,浙江
4,002531.SZ,严俊旭,严俊旭,朱彬,180250.9062,20050118,江苏


In [113]:
import tushare as ts
ts.get_notices("000001").head()

,title,type,date,url
0,平安银行：平安银行股份有限公司2020年年度权益分派实施公告2021-05-07,临时公告,2021-05-07,http://vip.stock.finance.sina.com.cn/corp/view...
1,平安银行：2021年第一季度报告全文,一季度报告,2021-04-21,http://vip.stock.finance.sina.com.cn/corp/view...
2,平安银行：一季报监事会决议公告,临时公告,2021-04-21,http://vip.stock.finance.sina.com.cn/corp/view...
3,平安银行：一季报董事会决议公告,临时公告,2021-04-21,http://vip.stock.finance.sina.com.cn/corp/view...
4,平安银行：2021年第一季度报告正文,一季度报告（摘要）,2021-04-21,http://vip.stock.finance.sina.com.cn/corp/view...


In [114]:
ts.guba_sina().head()

,title,ptime,rcounts
0,机构减仓或在战役性转移,05月07日 14:57,360.0
1,170家电子行业公司发布年度业绩预告,,0.0
2,券商开户专属通道：新客专享理财福利多多,,0.0
3,热门股票如何选择盘中介入点,05月07日 07:40,419.0
4,节后综合症再现 依然不乏结构性机会,05月07日 08:03,507.0


In [115]:
ts.get_hist_data("000001").head()

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
date,,,,,,,,,,,,,,
2021-05-07,23.67,24.30,24.05,23.39,802214.25,0.55,2.34,23.556,23.294,22.153,614633.11,766603.84,700531.86,0.41
2021-05-06,23.10,23.70,23.50,23.10,500295.19,0.21,0.90,23.334,23.058,22.034,548276.55,769859.69,680476.29,0.26
2021-04-30,23.35,23.49,23.29,23.01,561981.31,-0.30,-1.27,23.222,22.823,21.934,622701.05,831058.44,684515.71,0.29
2021-04-29,23.34,23.71,23.59,23.11,614836.88,0.24,1.03,23.222,22.520,21.859,674950.82,847860.13,683639.09,0.32
2021-04-28,23.29,23.45,23.35,22.78,593837.94,0.41,1.79,23.100,22.197,21.780,717922.23,859962.95,691748.65,0.31


In [116]:
ts.get_hist_data("000001",ktype="w").head()

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
date,,,,,,,,,,,,,,
2021-05-07,23.10,24.30,24.05,23.10,1302509.50,0.76,3.26,22.438,21.912,21.823,2882347.97,3893787.24,4866394.11,0.67
2021-04-30,23.87,24.23,23.29,22.78,3113505.25,0.00,0.00,21.928,21.645,21.538,3307222.47,4560954.59,5004209.48,1.60
2021-04-23,20.03,23.65,23.29,19.91,5197079.00,3.03,14.96,21.498,21.701,21.300,3544311.92,4492385.46,5036850.99,2.68
2021-04-16,21.51,21.51,20.26,19.81,3097949.00,-1.04,-4.88,20.934,21.754,21.100,3492326.92,4367200.06,5044014.37,1.60
2021-04-09,21.55,22.09,21.30,21.08,1700697.12,-0.20,-0.93,21.178,22.221,21.072,3922926.32,4865955.41,5078637.83,0.88


In [117]:
ts.get_hist_data("000001",ktype="m").head()

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
date,,,,,,,,,,,,,,
2021-05-07,23.10,24.30,24.05,23.10,1302509.5,0.76,3.26,22.764,20.090,17.241,17859144.9,20308849.85,20748658.73,0.67
2021-04-30,22.08,24.23,23.29,19.81,14234763.0,1.28,5.82,21.822,19.019,16.818,21521885.8,24854576.10,21952860.65,7.34
2021-03-31,21.54,23.49,22.01,20.28,23400600.0,0.63,2.95,21.112,17.970,16.361,22789929.6,25258599.60,22737767.70,12.06
2021-02-26,23.00,25.31,21.38,21.21,22432726.0,-1.71,-7.41,20.260,17.069,15.967,22277667.2,24135001.50,22393905.60,11.56
2021-01-29,19.10,23.54,23.09,17.80,27925126.0,3.75,19.39,19.018,16.324,15.587,22094764.0,23598847.10,22103348.45,14.39


In [118]:
ts.get_hist_data("000001",ktype="5").head()  # 5min (15,30,60)

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20,turnover
date,,,,,,,,,,,,,,
2021-05-07 15:00:00,24.01,24.03,24.01,23.98,14157.80,0.00,0.00,24.028,23.998,24.0555,13524.9,12686.8,11663.7,0.01
2021-05-07 14:55:00,24.03,24.04,24.01,23.96,14808.00,-0.02,-0.08,24.024,24.002,24.0630,12897.5,13060.2,11526.3,0.01
2021-05-07 14:50:00,24.08,24.10,24.02,23.96,20615.30,-0.06,-0.25,23.998,24.011,24.0670,12849.7,13332.0,11336.4,0.01
2021-05-07 14:45:00,24.01,24.08,24.08,24.01,9387.44,0.07,0.29,23.990,24.012,24.0680,10501.1,12905.3,11421.0,0.01
2021-05-07 14:40:00,24.00,24.03,24.02,23.95,8656.08,0.02,0.08,23.978,24.012,24.0655,11153.4,12820.4,11752.8,0.01


In [119]:
# sh上证指数； sz深圳成指； hs300沪深300； sz50上证50； zxb中小板指数； cyb创业板指数
ts.get_hist_data("cyb").head()

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


,open,high,close,low,volume,price_change,p_change,ma5,ma10,ma20,v_ma5,v_ma10,v_ma20
date,,,,,,,,,,,,,
2021-05-07,3021.87,3026.85,2910.41,2910.41,14489343.0,-104.40,-3.46,3023.724,2992.490,2898.924,16444300.8,16648376.3,15426893.45
2021-05-06,3059.58,3059.58,3014.81,2961.99,16552269.0,-76.59,-2.48,3038.848,2991.060,2894.984,16974533.4,16827430.9,15346704.55
2021-04-30,3040.65,3110.16,3091.40,3037.31,16606502.0,39.98,1.31,3029.766,2979.452,2886.855,17752532.8,17050663.6,15277921.70
2021-04-29,3053.74,3068.60,3051.42,3014.55,16647988.0,0.84,0.03,3010.384,2948.649,2873.055,17834241.6,16717391.8,15101499.00
2021-04-28,2990.70,3052.04,3050.58,2980.65,17925402.0,64.55,2.16,2987.978,2922.571,2858.409,17635713.4,16185039.8,14900826.45


### 2.股票数据获取（巨宽数据源）

In [40]:
from jqdatasdk import *

auth('18280180192', 'Tencent123')  # 请自行前往 https://www.joinquant.com/ 免费申请TOKEN

security = '000300.XSHG'
start_date = '2020-01-01'
end_date = '2020-03-01'

In [42]:
# security 股票代码
# frequency 时间粒度（1d=日）
# skip_paused 是否跳过缺失交易数据时间点
stock_price = get_price(security=security, start_date=start_date, end_date=end_date, frequency='1d',skip_paused=False)

stock_price.head()

,open,close,high,low,volume,money
2020-01-02,4121.35,4152.24,4172.66,4121.35,1.821168e+10,2.701055e+11
2020-01-03,4161.22,4144.96,4164.30,4131.86,1.428262e+10,2.152163e+11
2020-01-06,4120.52,4129.30,4170.64,4102.38,1.753100e+10,2.501821e+11
2020-01-07,4137.40,4160.23,4161.25,4135.10,1.394890e+10,1.963891e+11
2020-01-08,4139.63,4112.32,4149.81,4101.98,1.675858e+10,2.124063e+11
